In [ ]:
import matplotlib.pyplot as plt

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

import tqdm

import holodeck as holo
from holodeck import utils, plot
from holodeck.constants import YR

mpl.style.use('default')   # avoid dark backgrounds from dark theme vscode
%matplotlib inline
%config InlineBackend.figure_format = 'retina'


# Compare Simple Sample

In [ ]:
from holodeck.constants import YR, MSOL
import kalepy as kale

In [ ]:
dur1 = 5.0*YR/3.1557600
cad1 = .5*YR/3.1557600
fobs1 = utils.nyquist_freqs(dur1,cad1)
fobs_edges1 = utils.nyquist_freqs_edges(dur1,cad1)
# print('fobs=', fobs)
# print('fobs_edges=',fobs_edges)
print(f"Number of frequency bins: {fobs.size-1}")
print(f"  between [{fobs1[0]*YR:.2f}, {fobs1[-1]*YR:.2f}] 1/yr")
print(f"          [{fobs1[0]*1e9:.2f}, {fobs1[-1]*1e9:.2f}] nHz")
mtot=(1.0e6*MSOL/1.988409870698051, 1.0e8*MSOL/1.988409870698051, 3)
mrat=(1e-1, 1.0, 2)
redz=(1e-3, 1.0, 4)
sam1 = holo.sam.Semi_Analytic_Model(mtot=mtot, mrat=mrat, 
        redz=redz)
print('edges:', sam1.edges)

inside sam.gwb(Hard_GW)

In [ ]:
fobs_gw_edges1 = fobs_edges1
hard = holo.hardening.Hard_GW
fobs_gw_cents1 = kale.utils.midpoints(fobs_gw_edges1)
fobs_orb_edges1 = fobs_gw_edges1 / 2.0
fobs_orb_cents1 = fobs_gw_cents1 / 2.0

# dynamic_binary_number
edges1, dnum1 = sam1.dynamic_binary_number(hard, fobs_orb=fobs_orb_cents1)
edges1[-1] = fobs_orb_edges1

# integrate (multiply by bin volume) within each
# bin
number1 = utils._integrate_grid_differential_number(edges1, dnum1, freq=False)
number1 = number1 * np.diff(np.log(fobs_gw_edges1))

use_redz1 = sam1._redz_prime[:, :, :, np.newaxis] * np.ones_like(dnum1)


compare steps of 
* gravwaves._gws_from_number_grid_integrated_redz(edges, redz, number, realize=1, sum=True)
* single_sources.ss_gws_redz(edges, redz, number, realize=1, loudest = 1, params = False)

In [ ]:
gwb1 = holo.gravwaves._gws_from_number_grid_integrated_redz(edges1, use_redz1, number1, 
                                                           realize=1, sum=True)
hc_ss1, hc_bg1 = holo.single_sources.ss_gws_redz(edges1, use_redz1, number1, realize=1, loudest=100, params=False)
# hc_ss_tot = np.sqrt(
hc_tot1 = np.sqrt(np.sum(hc_ss1**2, axis=2)+ hc_bg1**2)

In [ ]:
print(np.array(edges1).shape)

In [ ]:
print(hc_bg1.squeeze())
print(gwb1.squeeze())
hc_tot1 = np.sqrt(np.sum(hc_ss1**2, axis=2)+ hc_bg1**2)
print(hc_tot1.shape)
print(gwb1.shape)
for rr in range(len(gwb1[0])):
    plt.plot(fobs1*YR, hc_tot1[:,rr]/gwb1[:,rr], label='hc_tot/gwb', alpha=0.25)

plt.xlabel('GW frequency (1/yr)')
plt.ylabel('Ratio hc_tot / gwb')

In [ ]:
plt.xlabel('GW frequency (1/yr)')
plt.ylabel('Strain $h_c$')

plt.loglog(fobs1*YR, gwb1[:], color='red', alpha=0.25, label='gwb')
plt.loglog(fobs1*YR, hc_tot1[:,0], color='b', alpha=0.25, label='hc_tot')
plt.legend()

### 2 Simple again. more louds

In [ ]:
hc_ss2, hc_bg2 = sam1.ss_gwb(fobs_edges1, realize=1, loudest=10)

In [ ]:
gwb2 = sam1.gwb(fobs_edges1, realize=1)

In [ ]:
print(hc_bg2.squeeze())
print(gwb2.squeeze())
hc_tot2 = np.sqrt(np.sum(hc_ss2**2, axis=2)+ hc_bg2**2)
print(hc_tot2.shape)
print(gwb2.shape)
plt.plot(fobs1*YR, hc_tot2[:,0]/gwb2[:], label='hc_tot/gwb')

In [ ]:
plt.xlabel('GW frequency (1/yr)')
plt.ylabel('Strain $h_c$')

plt.loglog(fobs1*YR, gwb2[:], color='red', alpha=0.25, label='gwb')
plt.loglog(fobs1*YR, hc_tot2[:,0], color='b', alpha=0.25, label='hc_tot')
plt.legend()

For the super simple example they match!!

### 3 Let's try full size SAM:

In [ ]:
dur3 = 10.0 *YR
cad3 = 0.2* YR
fobs3 = utils.nyquist_freqs(dur3,cad3) # obs gw freqs
fobs_edges3 = utils.nyquist_freqs_edges(dur3,cad3) # obs gw fres
sam3 = holo.sam.Semi_Analytic_Model()

In [ ]:
hc_ss3, hc_bg3 = sam3.ss_gwb(fobs_edges3, realize=30, loudest=10)
gwb3 = sam3.gwb(fobs_edges3, realize=30)

In [ ]:
# print(hc_bg3.squeeze())
# print(gwb3.squeeze())
hc_tot3 = np.sqrt(np.sum(hc_ss3**2, axis=2)+ hc_bg3**2)
print(hc_tot3.shape)
print(gwb3.shape)
for rr in range(len(gwb3[0])):
    plt.plot(fobs3*YR, hc_tot3[:,rr]/gwb3[:,rr], label='hc_tot/gwb', alpha=0.25)

plt.plot(fobs3*YR, np.mean(hc_tot3/gwb3, axis=1), color='k')

plt.xlabel('GW frequency (1/yr)')
plt.ylabel('Ratio hc_tot / gwb')
plt.ylim(0,3)

In [ ]:
plt.xlabel('GW frequency (1/yr)')
plt.ylabel('Strain $h_c$')

rr=0
plt.loglog(fobs3*YR, gwb3[:,rr], color='red', alpha=0.25, label='gwb')
plt.loglog(fobs3*YR, hc_tot3[:,rr], color='b', alpha=0.25, label='hc_tot')
for rr in range(1,len(gwb3[0])):
    plt.loglog(fobs3*YR, gwb3[:,rr], color='red', alpha=0.25)
    plt.loglog(fobs3*YR, hc_tot3[:,rr], color='b', alpha=0.25)
plt.legend()

For the super simple example they match!!

### 4 Let's try full size SAM, more reals:

In [ ]:
hc_ss4, hc_bg4 = sam3.ss_gwb(fobs_edges3, realize=100, loudest=10)
gwb4 = sam3.gwb(fobs_edges3, realize=100)

In [ ]:
# print(hc_bg3.squeeze())
# print(gwb3.squeeze())
hc_tot4 = np.sqrt(np.sum(hc_ss4**2, axis=2)+ hc_bg4**2)
print(hc_tot4.shape)
print(gwb4.shape)
for rr in range(len(gwb4[0])):
    plt.plot(fobs3*YR, hc_tot4[:,rr]/gwb4[:,rr], label='hc_tot/gwb', alpha=0.25)

plt.plot(fobs3*YR, np.mean(hc_tot4/gwb4, axis=1), color='k')

plt.xlabel('GW frequency (1/yr)')
plt.ylabel('Ratio hc_tot / gwb')
plt.ylim(0,3)

In [ ]:
plt.xlabel('GW frequency (1/yr)')
plt.ylabel('Strain $h_c$')

rr=0
plt.loglog(fobs3*YR, gwb4[:,rr], color='red', alpha=0.25, label='gwb')
plt.loglog(fobs3*YR, hc_tot4[:,rr], color='b', alpha=0.25, label='hc_tot')
for rr in range(1,len(gwb4[0])):
    plt.loglog(fobs3*YR, gwb4[:,rr], color='red', alpha=0.25)
    plt.loglog(fobs3*YR, hc_tot4[:,rr], color='b', alpha=0.25)
plt.legend()

In [ ]:
sum1 = np.sqrt(np.sum(hc_bg4[:, :, np.newaxis]**2 + hc_ss4**2, axis=-1))
print(sum1.shape)
sum2 = np.sqrt(np.sum(hc_ss4**2, axis=2) + hc_bg4**2)
# why are these not the exact same??
plt.plot(sum1[:,0], sum2[:,0])
plt.xlabel('sum1 = $\sqrt{\sum_\mathrm{axis=-1}(hc_{bg4}[:, :, np.newaxis]^2 + hc_{ss4}^2})$')
plt.ylabel('sum2 = $\sqrt{\sum_L(hc_{ss4}^2) + hc_{bg4}^2}$')

In [ ]:
fig, axes = plot.figax(
    figsize=[8, 8], nrows=2, hspace=0.4,
    xlabel=plot.LABEL_GW_FREQUENCY_YR, ylabel=plot.LABEL_CHARACTERISTIC_STRAIN
)


xx = fobs3 * YR

# y1 = data['hc_bg']
# y1 = np.sqrt(np.sum(hc_bg4[:, :, np.newaxis]**2 + hc_ss4**2, axis=-1))
y1=hc_tot4
print(y1.shape)
y2 = gwb4

labels = ['SS', 'GWB']
for jj, yy in enumerate([y1, y2]):
    med, *conf = np.percentile(yy, [50, 25, 75], axis=-1)
    cc, = axes[0].plot(xx, med, alpha=0.5, label=labels[jj])
    cc = cc.get_color()
    axes[0].fill_between(xx, *conf, color=cc, alpha=0.1)

axes[1].plot(xx, np.median(y1, axis=-1)/np.median(y2, axis=-1), 'k--', label='SS/GWB median')
axes[1].plot(xx, np.mean(y1, axis=-1)/np.mean(y2, axis=-1), 'k--', label='SS/GWB mean')
        

axes[1].set_ylabel('ratio')
for ax in axes:
    ax.legend()
    plot._twin_hz(ax)
    
plt.show()

# Compare Old (noredz)

In [ ]:
def run_single_params_model(space, params, fobs_edges, nreals=100, nloudest=5):
    param_names = space.param_names
    log = space._log
    
    pars = {name: params[pp] for pp, name in enumerate(param_names)}

    log.info(f"Loading sam and hard from {params=}")
    sam, hard = space.model_for_params(pars, sam_shape=space.sam_shape)

    fobs_cents = utils.midpoints(fobs_edges, log=False)
    fobs_orb_edges = fobs_edges / 2.0
    fobs_orb_cents = fobs_cents / 2.0

    log.info(f"Calculating `dynamic_binary_number` for {fobs_cents.size} freqs")
    edges, dnum = sam.dynamic_binary_number(hard, fobs_orb=fobs_orb_cents)
    edges[-1] = fobs_orb_edges

    # integrate for number
    log.debug(f"integrating to get number of binaries")
    number = utils._integrate_grid_differential_number(edges, dnum, freq=False)
    number = number * np.diff(np.log(fobs_edges))

    log.debug(f"calculating strains and sources for shape ({fobs_cents.size}, {nreals}), {nloudest=}")
    hc_ss, hc_bg, sspar, bgpar = holo.single_sources.ss_gws(
        edges, number, realize=nreals, loudest=nloudest, params=True
    )

    # use_redz = sam._redz_final
    # gwb = holo.gravwaves._gws_from_number_grid_integrated(edges, use_redz, number, nreals)

    gwb = holo.gravwaves._gws_from_number_grid_integrated(edges, number, nreals)

    data = dict(
        fobs=fobs_cents, fobs_edges=fobs_edges,
        gwb=gwb, hc_ss=hc_ss, hc_bg=hc_bg, sspar=sspar, bgpar=bgpar
    )
    return data


def run_params_subspace(space, params, fobs_edges, **kwargs):
    log = space._log
    param_names = space.param_names
    npars = len(param_names)
    params = np.asarray(params)

    # Make sure shapes look good
    if params.ndim != 2 or params.shape[1] != npars:
        err = f"`params` shape {np.shape(params)} does not match {npars} parameters: {param_names}!"
        log.exception(err)
        raise RuntimeError(err)

    # Make sure all parameters are within excepted bounds
    for pp, name in enumerate(param_names):
        dist = space._dists[pp]

        bads = (params[:, pp] <= dist.extrema[0]) | (dist.extrema[1] <= params[:, pp])
        if np.any(bads):
            err = f"`params` {pp} {name} are outside of extrema [{dist.extrema[0]:.8e}, {dist.extrema[1]:.8e}]!"
            err += f"  {utils.stats(params[:, pp])}"
            log.exception(err)
            raise ValueError(err)

    num_samps = params.shape[0]

    all_data = []
    for ii in tqdm.trange(num_samps):
        data = run_single_params_model(space, params[ii], fobs_edges, **kwargs)
        all_data.append(data)

    return all_data


NUM_FBINS = 14
PTA_DUR = 16.03     # [yrs]
SHAPE = 40
NREALS = 50

space_class = holo.param_spaces.PS_Broad_Uniform_02B
space = space_class(holo.log, 0, SHAPE, None)

dur = PTA_DUR * YR
pta_cad = dur / (2 * NUM_FBINS)
fobs_edges = holo.utils.nyquist_freqs_edges(dur, pta_cad)

draw_params = [
    [ 4.57784231, -1.51291368, 10.90450461,  8.85735088,  0.52998213],
    [ 6.96691128, -2.38054765, 11.08484247,  9.29421616,  0.45471499],
    [ 0.37454247, -2.10501603, 11.62087475,  8.84882612,  0.09778727]
]

# Run full set of `draw_params`
# all_data = run_params_subspace(space, draw_params[:4], fobs_edges, nreals=NREALS)

single_data = run_single_params_model(space, draw_params[0], fobs_edges, nreals=NREALS)


In [ ]:
fig, axes = plot.figax(
    figsize=[8, 8], nrows=2, hspace=0.4,
    xlabel=plot.LABEL_GW_FREQUENCY_YR, ylabel=plot.LABEL_CHARACTERISTIC_STRAIN
)


xx = single_data['fobs'] * YR

# y1 = data['hc_bg']
y1 = np.sqrt(np.sum(single_data['hc_bg'][:, :, np.newaxis]**2 + single_data['hc_ss']**2, axis=-1))
y2 = single_data['gwb']

labels = ['SS', '2x GWB']
for jj, yy in enumerate([y1, 2*y2]):
    med, *conf = np.percentile(yy, [50, 25, 75], axis=-1)
    cc, = axes[0].plot(xx, med, alpha=0.5, label=labels[jj])
    cc = cc.get_color()
    axes[0].fill_between(xx, *conf, color=cc, alpha=0.1)

axes[1].plot(xx, np.median(y1, axis=-1)/np.median(y2, axis=-1), 'k--', label='SS/GWB')
        

axes[1].set_ylabel('ratio')
for ax in axes:
    ax.legend()
    plot._twin_hz(ax)
    
plt.show()

# Compare New redz funcs

In [ ]:
def run_single_params_model_new(space, params, fobs_edges, nreals=100, nloudest=5):
    param_names = space.param_names
    log = space._log
    
    pars = {name: params[pp] for pp, name in enumerate(param_names)}

    log.info(f"Loading sam and hard from {params=}")
    sam, hard = space.model_for_params(pars, sam_shape=space.sam_shape)

    fobs_cents = utils.midpoints(fobs_edges, log=False)
    fobs_orb_edges = fobs_edges / 2.0
    fobs_orb_cents = fobs_cents / 2.0

    # log.info(f"Calculating `dynamic_binary_number` for {fobs_cents.size} freqs")
    # edges, dnum = sam.dynamic_binary_number(hard, fobs_orb=fobs_orb_cents)
    # edges[-1] = fobs_orb_edges

    # # integrate for number
    # log.debug(f"integrating to get number of binaries")
    # number = utils._integrate_grid_differential_number(edges, dnum, freq=False)
    # number = number * np.diff(np.log(fobs_edges))

    log.debug(f"calculating strains and sources for shape ({fobs_cents.size}, {nreals}), {nloudest=}")
    hc_ss, hc_bg, sspar, bgpar = sam.ss_gwb(fobs_edges, hard=hard, realize=nreals, loudest=nloudest, params=True)

    # use_redz = sam._redz_final
    # gwb = holo.gravwaves._gws_from_number_grid_integrated(edges, use_redz, number, nreals)

    gwb = sam.gwb(fobs_edges, hard=hard, realize=nreals)

    data = dict(
        fobs=fobs_cents, fobs_edges=fobs_edges,
        gwb=gwb, hc_ss=hc_ss, hc_bg=hc_bg, sspar=sspar, bgpar=bgpar
    )
    return data


def run_params_subspace(space, params, fobs_edges, **kwargs):
    log = space._log
    param_names = space.param_names
    npars = len(param_names)
    params = np.asarray(params)

    # Make sure shapes look good
    if params.ndim != 2 or params.shape[1] != npars:
        err = f"`params` shape {np.shape(params)} does not match {npars} parameters: {param_names}!"
        log.exception(err)
        raise RuntimeError(err)

    # Make sure all parameters are within excepted bounds
    for pp, name in enumerate(param_names):
        dist = space._dists[pp]

        bads = (params[:, pp] <= dist.extrema[0]) | (dist.extrema[1] <= params[:, pp])
        if np.any(bads):
            err = f"`params` {pp} {name} are outside of extrema [{dist.extrema[0]:.8e}, {dist.extrema[1]:.8e}]!"
            err += f"  {utils.stats(params[:, pp])}"
            log.exception(err)
            raise ValueError(err)

    num_samps = params.shape[0]

    all_data = []
    for ii in tqdm.trange(num_samps):
        data = run_single_params_model(space, params[ii], fobs_edges, **kwargs)
        all_data.append(data)

    return all_data


NUM_FBINS = 14
PTA_DUR = 16.03     # [yrs]
SHAPE = 40
NREALS = 50

space_class = holo.param_spaces.PS_Broad_Uniform_02B
space = space_class(holo.log, 0, SHAPE, None)

dur = PTA_DUR * YR
pta_cad = dur / (2 * NUM_FBINS)
fobs_edges = holo.utils.nyquist_freqs_edges(dur, pta_cad)

draw_params = [
    [ 4.57784231, -1.51291368, 10.90450461,  8.85735088,  0.52998213],
    [ 6.96691128, -2.38054765, 11.08484247,  9.29421616,  0.45471499],
    [ 0.37454247, -2.10501603, 11.62087475,  8.84882612,  0.09778727]
]

# Run full set of `draw_params`
# all_data = run_params_subspace(space, draw_params[:4], fobs_edges, nreals=NREALS)

single_data = run_single_params_model_new(space, draw_params[0], fobs_edges, nreals=NREALS)


In [ ]:
fig, axes = plot.figax(
    figsize=[8, 8], nrows=2, hspace=0.4,
    xlabel=plot.LABEL_GW_FREQUENCY_YR, ylabel=plot.LABEL_CHARACTERISTIC_STRAIN
)


xx = single_data['fobs'] * YR

# y1 = data['hc_bg']
y1 = np.sqrt(np.sum(single_data['hc_bg'][:, :, np.newaxis]**2 + single_data['hc_ss']**2, axis=-1))
y2 = single_data['gwb']

labels = ['SS', '2x GWB']
for jj, yy in enumerate([y1, 2*y2]):
    med, *conf = np.percentile(yy, [50, 25, 75], axis=-1)
    cc, = axes[0].plot(xx, med, alpha=0.5, label=labels[jj])
    cc = cc.get_color()
    axes[0].fill_between(xx, *conf, color=cc, alpha=0.1)

axes[1].plot(xx, np.median(y1, axis=-1)/np.median(y2, axis=-1), 'k--', label='SS/GWB')
        

axes[1].set_ylabel('ratio')
for ax in axes:
    ax.legend()
    plot._twin_hz(ax)
    
plt.show()